In [1]:
import pandas as pd
from utils.scraping_utils import get_html, parrarelize_processes
from wrappers import extract_articles
from representatives.collection import extraction_functions


In [4]:
rep_df = pd.read_csv('data\chosen_representatives.csv')
rep_df.tail(45)

,state,party,district,name,office_room,phone,committee,page_link
40,Mississippi,D,2nd,"Thompson, Bennie",2466 RHOB,(202) 225-5876,Homeland Security,https://benniethompson.house.gov/
41,Mississippi,R,1st,"Kelly, Trent",2243 RHOB,(202) 225-4306,Agriculture|Armed Services|Intelligence,https://trentkelly.house.gov/
42,Missouri,D,1st,"Bush, Cori",2463 RHOB,(202) 225-2406,Oversight and Accountability|Judiciary,https://bush.house.gov
43,Missouri,R,2nd,"Wagner, Ann",2350 RHOB,(202) 225-1621,Financial Services|Foreign Affairs,https://wagner.house.gov
44,Montana,R,1st,"Zinke, Ryan",512 CHOB,(202) 225-5628,Appropriations|Foreign Affairs,https://zinke.house.gov
45,Nebraska,R,1st,"Flood, Mike",343 CHOB,(202) 225-4806,Financial Services,https://flood.house.gov/
46,Nevada,D,1st,"Titus, Dina",2464 RHOB,(202) 225-5965,Foreign Affairs|Transportation and Infrastructure,https://titus.house.gov/
47,Nevada,R,2nd,"Amodei, Mark",104 CHOB,(202) 225-6155,Appropriations,https://amodei.house.gov
48,New Hampshire,D,1st,"Pappas, Chris",452 CHOB,(202) 225-5456,Transportation and Infrastructure|Small Busine...,https://pappas.house.gov
49,New Jersey,D,1st,"Norcross, Donald",2427 RHOB,(202) 225-6501,Armed Services|Education and the Workforce,https://norcross.house.gov


In [5]:
df1 = pd.read_csv('data/articles.csv', index_col=0)
df1

,issue,title,date,text,representative_name,representative_state,representative_party,representative_committee
idx,,,,,,,,
0,Economy,Rep. Sewell Delivers $1.6 Million in Federal F...,"October 11, 2024","Birmingham, AL – Today, U.S. Rep. Terri Sewell...","Sewell, Terri",Alabama,D,Armed Services|House Administration|Joint Comm...
1,Economy,Rep. Sewell Announces 13th Annual Job Fair,"July 11, 2024",The 2024 Job Fair will be held at the Bessemer...,"Sewell, Terri",Alabama,D,Armed Services|House Administration|Joint Comm...
2,Economy,Rep. Sewell Announces $16.9 Million from Presi...,"July 9, 2024","Montgomery, AL — Today, U.S. Rep. Terri Sewell...","Sewell, Terri",Alabama,D,Armed Services|House Administration|Joint Comm...
3,Economy,Rep. Sewell Announces $14+ Million in Departme...,"June 24, 2024","Fairfield, AL — Today, U.S. Rep. Terri Sewell ...","Sewell, Terri",Alabama,D,Armed Services|House Administration|Joint Comm...
4,Economy,Rep. Sewell Hosts Seniors Resource Roundtable ...,"June 18, 2024",Sewell highlighted how the Inflation Reduction...,"Sewell, Terri",Alabama,D,Armed Services|House Administration|Joint Comm...
...,...,...,...,...,...,...,...,...
13324,Energy,Scalise to host 2013 American Energy Tour Thur...,"February 20, 2013","Wednesday, February 20, 2013Scalise to host 20...","Scalise, Steve",Louisiana,R,NaN
13325,Energy,Scalise meets with new Director of Bureau of S...,"December 16, 2011","Washington, D.C. – Congressman Steve Scalise t...","Scalise, Steve",Louisiana,R,NaN
13326,Energy,Scalise: Obama lease plan says no to American...,"June 28, 2012","Washington, D.C. – Congressman Steve Scalise t...","Scalise, Steve",Louisiana,R,NaN


In [6]:
df2 = pd.read_csv('data/articles2.csv', index_col=0)
df2

,issue,title,date,text,representative_name,representative_state,representative_party,representative_committee
0,Health,Rep. Scholten Responds to Emergency Abortion S...,"June 27, 2024","WASHINGTON – Today, the Supreme Court dismisse...","Scholten, Hillary",Michigan,D,Transportation and Infrastructure|Small Business
1,Veterans,Rep. Scholten Introduces Bill to Rename Post O...,"August 21, 2024",WASHINGTON – Congresswoman Hillary Scholten in...,"Scholten, Hillary",Michigan,D,Transportation and Infrastructure|Small Business
2,Energy,Rep. Scholten Secures Over $500K for Energy In...,"September 19, 2024","WASHINGTON – Today, U.S. Representative Hillar...","Scholten, Hillary",Michigan,D,Transportation and Infrastructure|Small Business
3,Health,Representative Hillary Scholten addresses her ...,"January 17, 2023",H.R. 26H.R. 26 does not actually add anything ...,"Scholten, Hillary",Michigan,D,Transportation and Infrastructure|Small Business
4,Economy,Rep. Scholten Leads Small Business Committee D...,"October 26, 2023","WASHINGTON, D.C. — Today, Rep. Hillary Scholte...","Scholten, Hillary",Michigan,D,Transportation and Infrastructure|Small Business
...,...,...,...,...,...,...,...,...
4078,America's Workers,A Call for Fairness as We Observe Women’s Hist...,"March 1, 2016","Washington, DC-Rep. Donald Norcross (NJ-01) to...","Norcross, Donald",New Jersey,D,Armed Services|Education and the Workforce
4079,America's Workers,"""Make It In America""- Making It In South Jersey","July 19, 2015","Camden, NJ - Friday, Rep. Donald Norcross (NJ-...","Norcross, Donald",New Jersey,D,Armed Services|Education and the Workforce
4080,America's Workers,U.S. House Passes Long-Term Transportation Bill,"November 5, 2015","Washington, DC- Rep. Donald Norcross (NJ-01) t...","Norcross, Donald",New Jersey,D,Armed Services|Education and the Workforce
4081,America's Workers,Local Heroes Get Help Keeping Communities Safe,"August 12, 2015","Washington, DC - Congressman Donald Norcross (...","Norcross, Donald",New Jersey,D,Armed Services|Education and the Workforce


In [7]:
df_sum = pd.concat([df1, df2])
df_sum

,issue,title,date,text,representative_name,representative_state,representative_party,representative_committee
0,Economy,Rep. Sewell Delivers $1.6 Million in Federal F...,"October 11, 2024","Birmingham, AL – Today, U.S. Rep. Terri Sewell...","Sewell, Terri",Alabama,D,Armed Services|House Administration|Joint Comm...
1,Economy,Rep. Sewell Announces 13th Annual Job Fair,"July 11, 2024",The 2024 Job Fair will be held at the Bessemer...,"Sewell, Terri",Alabama,D,Armed Services|House Administration|Joint Comm...
2,Economy,Rep. Sewell Announces $16.9 Million from Presi...,"July 9, 2024","Montgomery, AL — Today, U.S. Rep. Terri Sewell...","Sewell, Terri",Alabama,D,Armed Services|House Administration|Joint Comm...
3,Economy,Rep. Sewell Announces $14+ Million in Departme...,"June 24, 2024","Fairfield, AL — Today, U.S. Rep. Terri Sewell ...","Sewell, Terri",Alabama,D,Armed Services|House Administration|Joint Comm...
4,Economy,Rep. Sewell Hosts Seniors Resource Roundtable ...,"June 18, 2024",Sewell highlighted how the Inflation Reduction...,"Sewell, Terri",Alabama,D,Armed Services|House Administration|Joint Comm...
...,...,...,...,...,...,...,...,...
4078,America's Workers,A Call for Fairness as We Observe Women’s Hist...,"March 1, 2016","Washington, DC-Rep. Donald Norcross (NJ-01) to...","Norcross, Donald",New Jersey,D,Armed Services|Education and the Workforce
4079,America's Workers,"""Make It In America""- Making It In South Jersey","July 19, 2015","Camden, NJ - Friday, Rep. Donald Norcross (NJ-...","Norcross, Donald",New Jersey,D,Armed Services|Education and the Workforce
4080,America's Workers,U.S. House Passes Long-Term Transportation Bill,"November 5, 2015","Washington, DC- Rep. Donald Norcross (NJ-01) t...","Norcross, Donald",New Jersey,D,Armed Services|Education and the Workforce
4081,America's Workers,Local Heroes Get Help Keeping Communities Safe,"August 12, 2015","Washington, DC - Congressman Donald Norcross (...","Norcross, Donald",New Jersey,D,Armed Services|Education and the Workforce


In [45]:
# df_sum.drop('index', axis=1,inplace=True)


In [8]:
df_sum.reset_index(inplace=True, drop=True)
df_sum.to_csv('data/articles.csv')

In [9]:
df_sum['representative_name'].value_counts()

representative_name
Bishop, Sanford              3573
Miller-Meeks, Mariannette    1488
LaMalfa, Doug                1026
Larson, John                  941
Radewagen, Aumua Amata        921
Norcross, Donald              909
Boebert, Lauren               873
Davids, Sharice               828
Scalise, Steve                773
Craig, Angie                  738
Harris, Andy                  620
Ruppersberger, C.             556
Pocan, Mark                   537
Bost, Mike                    489
Mrvan, Frank                  447
Sewell, Terri                 440
Mann, Tracey                  373
Flood, Mike                   346
Pappas, Chris                 302
Soto, Darren                  297
Wagner, Ann                   261
Zinke, Ryan                   229
Comer, James                  154
Carl, Jerry                    85
Gaetz, Matt                    84
Amodei, Mark                   77
Scholten, Hillary              45
Name: count, dtype: int64

In [10]:
df_sum['issue'].value_counts()


issue
Education                         1009
Economy                            819
Veterans                           793
Delivering Results                 744
Health Care                        593
                                  ... 
Law Enforcement                      2
Jobs And The Economy                 2
Native American Communities          2
Program Awards & Announcements       1
Spending Cuts & Debt                 1
Name: count, Length: 152, dtype: int64

In [11]:
df_sum['representative_party'].value_counts()


representative_party
D    9613
R    7799
Name: count, dtype: int64

In [12]:
df_sum

,issue,title,date,text,representative_name,representative_state,representative_party,representative_committee
0,Economy,Rep. Sewell Delivers $1.6 Million in Federal F...,"October 11, 2024","Birmingham, AL – Today, U.S. Rep. Terri Sewell...","Sewell, Terri",Alabama,D,Armed Services|House Administration|Joint Comm...
1,Economy,Rep. Sewell Announces 13th Annual Job Fair,"July 11, 2024",The 2024 Job Fair will be held at the Bessemer...,"Sewell, Terri",Alabama,D,Armed Services|House Administration|Joint Comm...
2,Economy,Rep. Sewell Announces $16.9 Million from Presi...,"July 9, 2024","Montgomery, AL — Today, U.S. Rep. Terri Sewell...","Sewell, Terri",Alabama,D,Armed Services|House Administration|Joint Comm...
3,Economy,Rep. Sewell Announces $14+ Million in Departme...,"June 24, 2024","Fairfield, AL — Today, U.S. Rep. Terri Sewell ...","Sewell, Terri",Alabama,D,Armed Services|House Administration|Joint Comm...
4,Economy,Rep. Sewell Hosts Seniors Resource Roundtable ...,"June 18, 2024",Sewell highlighted how the Inflation Reduction...,"Sewell, Terri",Alabama,D,Armed Services|House Administration|Joint Comm...
...,...,...,...,...,...,...,...,...
17407,America's Workers,A Call for Fairness as We Observe Women’s Hist...,"March 1, 2016","Washington, DC-Rep. Donald Norcross (NJ-01) to...","Norcross, Donald",New Jersey,D,Armed Services|Education and the Workforce
17408,America's Workers,"""Make It In America""- Making It In South Jersey","July 19, 2015","Camden, NJ - Friday, Rep. Donald Norcross (NJ-...","Norcross, Donald",New Jersey,D,Armed Services|Education and the Workforce
17409,America's Workers,U.S. House Passes Long-Term Transportation Bill,"November 5, 2015","Washington, DC- Rep. Donald Norcross (NJ-01) t...","Norcross, Donald",New Jersey,D,Armed Services|Education and the Workforce
17410,America's Workers,Local Heroes Get Help Keeping Communities Safe,"August 12, 2015","Washington, DC - Congressman Donald Norcross (...","Norcross, Donald",New Jersey,D,Armed Services|Education and the Workforce
